# 1.1 Document Ingestion Basics - Ungraph

Este notebook cubre la fase **EXTRACT** del patrón ETI: cómo cargar documentos desde diferentes formatos al sistema.

## Objetivos

1. **Configurar Ungraph** - Conexión a Neo4j y configuración básica
2. **Cargar documentos** - Markdown, TXT, Word, PDF
3. **Verificar carga** - Validar que los documentos se cargaron correctamente
4. **Obtener recomendaciones** - Estrategias de chunking recomendadas

## Formatos Soportados

- ✅ **Markdown (.md)** - Documentos estructurados
- ✅ **Texto plano (.txt)** - Con detección automática de encoding
- ✅ **Word (.docx)** - Documentos de Microsoft Word
- ✅ **PDF (.pdf)** - Usando IBM Docling para mejor extracción

**Referencias:**
- [Guía de Ingesta](../../docs/guides/ingestion.md)
- [API Pública](../../docs/api/public-api.md)


In [2]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
            return

# Agregar carpetas necesarias al path
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")


In [3]:
# Importar librerías necesarias
import sys
from pathlib import Path

# Importar handlers
from src.utils.handlers import find_in_project

# Importar ungraph
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

print(f"📦 Ungraph version: {ungraph.__version__}")


✅ Ungraph importado desde src/ (modo desarrollo)
📦 Ungraph version: 0.1.0


## Parte 1: Configuración Inicial

Primero configuramos la conexión a Neo4j. Puedes usar variables de entorno o configuración programática.


In [4]:
# Configurar Ungraph
# Opción 1: Usar variables de entorno (recomendado)
# export NEO4J_URI="bolt://localhost:7687"
# export NEO4J_PASSWORD="tu_contraseña"

# Opción 2: Configurar programáticamente
ungraph.configure(
    neo4j_uri="bolt://localhost:7687",
    neo4j_user="neo4j",
    neo4j_password="Ungraph22",  # ⚠️ CAMBIAR: Usa tu contraseña real
    neo4j_database="neo4j",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

print("✅ Configuración completada")
print("💡 Si obtienes AuthError, verifica que Neo4j esté corriendo y las credenciales sean correctas")


✅ Configuración completada
💡 Si obtienes AuthError, verifica que Neo4j esté corriendo y las credenciales sean correctas


## Parte 2: Encontrar Archivos de Datos

Localicemos los archivos de ejemplo disponibles.


In [5]:
# Encontrar la carpeta data
data_path = find_in_project(
    target="data",
    search_type="folder",
    project_root=None
)

print(f"📁 Carpeta de datos: {data_path}")

# Listar archivos disponibles
files = list(data_path.glob("*"))
print(f"\n📄 Archivos disponibles:")
for file in files:
    if file.is_file():
        size_kb = file.stat().st_size / 1024
        print(f"  - {file.name} ({size_kb:.1f} KB)")


📁 Carpeta de datos: D:\projects\Ungraph\src\data

📄 Archivos disponibles:
  - 110225.md (6.6 KB)
  - AnnyLetter.txt (17.8 KB)
  - peach-et-al-2014-qsar-modeling-of-imbalanced-high-throughput-screening-data-in-pubchem.pdf (447.4 KB)
  - Usar símboles de silencio de corchea.docx (25.2 KB)


## Parte 3: Ingerir Documentos por Formato

Ahora ingerimos documentos de diferentes formatos. Cada formato tiene sus características específicas.


### 3.1 Ingerir Markdown (.md)


In [6]:
# Ingerir archivo Markdown
markdown_file = data_path / "110225.md"

if markdown_file.exists():
    print(f"📥 Ingiriendo: {markdown_file.name}")
    chunks = ungraph.ingest_document(
        markdown_file,
        chunk_size=1000,
        chunk_overlap=200,
        clean_text=True
    )
    print(f"✅ Documento ingerido exitosamente!")
    print(f"   Total de chunks: {len(chunks)}")
    print(f"   Chunks con embeddings: {sum(1 for c in chunks if c.embeddings)}")
    
    # Mostrar información de algunos chunks
    print(f"\n📄 Primeros 2 chunks:")
    for i, chunk in enumerate(chunks[:2], 1):
        print(f"\n   Chunk {i}:")
        print(f"   - ID: {chunk.id[:50]}...")
        print(f"   - Contenido: {chunk.page_content[:100]}...")
        print(f"   - Consecutivo: {chunk.chunk_id_consecutive}")
        if chunk.embeddings:
            print(f"   - Embeddings: {len(chunk.embeddings)} dimensiones")
else:
    print(f"⚠️  Archivo no encontrado: {markdown_file}")


📥 Ingiriendo: 110225.md


2025-12-25 16:04:38,939 - INFO - CUDA no disponible, usando CPU para embeddings.
2025-12-25 16:04:38,945 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-12-25 16:04:41,967 - INFO - Embedding service initialized with model: sentence-transformers/all-MiniLM-L6-v2
2025-12-25 16:04:41,968 - INFO - Loading spaCy model: en_core_web_sm
2025-12-25 16:04:43,404 - INFO - spaCy model loaded successfully
2025-12-25 16:04:43,405 - INFO - Starting document ingestion: D:\projects\Ungraph\src\data\110225.md
2025-12-25 16:04:43,408 - INFO - Using pattern: FILE_PAGE_CHUNK
2025-12-25 16:04:43,409 - INFO - Step 1: Loading document
2025-12-25 16:04:43,410 - INFO - Cargando archivo Markdown: D:\projects\Ungraph\src\data\110225.md
2025-12-25 16:04:47,492 - INFO - Starting text cleaning.
2025-12-25 16:04:47,494 - INFO - Text cleaning completed.
2025-12-25 16:04:47,495 - INFO - Archivo cargado exitosamente. Documentos generados: 1
2025-12-25 16:04:47,496 - INFO - Step 

✅ Documento ingerido exitosamente!
   Total de chunks: 9
   Chunks con embeddings: 9

📄 Primeros 2 chunks:

   Chunk 1:
   - ID: 110225.md_bd7bc8ad-1c65-4bfb-8dfc-47b05ff94ca9...
   - Contenido: Incluso es lo primero que se abrio, quiza lo que me falta en este momento es la palabra, y necesite ...
   - Consecutivo: 1
   - Embeddings: 384 dimensiones

   Chunk 2:
   - ID: 110225.md_16ed4563-a987-47eb-a361-d00cffac4952...
   - Contenido: quiero narrar como estoy desarrollando la vision de mi mismo, la que quiero para mi, el humano que q...
   - Consecutivo: 2
   - Embeddings: 384 dimensiones


### 3.2 Ingerir Texto Plano (.txt)

Los archivos de texto tienen detección automática de encoding (UTF-8, Windows-1252, Latin-1, etc.).


In [7]:
# Ingerir archivo de texto
txt_file = data_path / "AnnyLetter.txt"

if txt_file.exists():
    print(f"📥 Ingiriendo: {txt_file.name}")
    chunks_txt = ungraph.ingest_document(
        txt_file,
        chunk_size=500,
        chunk_overlap=100,
        clean_text=True
    )
    print(f"✅ Documento ingerido exitosamente!")
    print(f"   Total de chunks: {len(chunks_txt)}")
else:
    print(f"⚠️  Archivo no encontrado: {txt_file}")


2025-12-25 16:04:58,507 - INFO - CUDA no disponible, usando CPU para embeddings.
2025-12-25 16:04:58,511 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


📥 Ingiriendo: AnnyLetter.txt


2025-12-25 16:05:00,898 - INFO - Embedding service initialized with model: sentence-transformers/all-MiniLM-L6-v2
2025-12-25 16:05:00,900 - INFO - Loading spaCy model: en_core_web_sm
2025-12-25 16:05:01,432 - INFO - spaCy model loaded successfully
2025-12-25 16:05:01,433 - INFO - Starting document ingestion: D:\projects\Ungraph\src\data\AnnyLetter.txt
2025-12-25 16:05:01,434 - INFO - Using pattern: FILE_PAGE_CHUNK
2025-12-25 16:05:01,435 - INFO - Step 1: Loading document
2025-12-25 16:05:01,436 - INFO - Cargando archivo de texto: D:\projects\Ungraph\src\data\AnnyLetter.txt
2025-12-25 16:05:01,463 - INFO - Codificación detectada por chardet: iso-8859-1 (confianza: 73.00%)
2025-12-25 16:05:01,464 - INFO - Codificación detectada: iso-8859-1
2025-12-25 16:05:01,466 - INFO - Archivo cargado exitosamente con codificación: iso-8859-1
2025-12-25 16:05:01,468 - INFO - Starting text cleaning.
2025-12-25 16:05:01,471 - INFO - Text cleaning completed.
2025-12-25 16:05:01,472 - INFO - Archivo carga

✅ Documento ingerido exitosamente!
   Total de chunks: 45


### 3.3 Ingerir Word (.docx)

Los documentos Word se procesan manteniendo la estructura y formato.


In [8]:
# Ingerir archivo Word
docx_file = data_path / "Usar símboles de silencio de corchea.docx"

if docx_file.exists():
    print(f"📥 Ingiriendo: {docx_file.name}")
    chunks_docx = ungraph.ingest_document(
        docx_file,
        chunk_size=1000,
        chunk_overlap=200,
        clean_text=True
    )
    print(f"✅ Documento ingerido exitosamente!")
    print(f"   Total de chunks: {len(chunks_docx)}")
    
    # Mostrar metadatos específicos de Word
    if chunks_docx:
        print(f"\n📋 Metadatos del primer chunk:")
        for key, value in list(chunks_docx[0].metadata.items())[:5]:
            print(f"   - {key}: {value}")
else:
    print(f"⚠️  Archivo no encontrado: {docx_file}")


2025-12-25 16:05:29,348 - INFO - CUDA no disponible, usando CPU para embeddings.
2025-12-25 16:05:29,351 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


📥 Ingiriendo: Usar símboles de silencio de corchea.docx


2025-12-25 16:05:31,760 - INFO - Embedding service initialized with model: sentence-transformers/all-MiniLM-L6-v2
2025-12-25 16:05:31,761 - INFO - Loading spaCy model: en_core_web_sm
2025-12-25 16:05:32,209 - INFO - spaCy model loaded successfully
2025-12-25 16:05:32,210 - INFO - Starting document ingestion: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx
2025-12-25 16:05:32,210 - INFO - Using pattern: FILE_PAGE_CHUNK
2025-12-25 16:05:32,211 - INFO - Step 1: Loading document
2025-12-25 16:05:32,212 - INFO - Cargando archivo Word: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx
2025-12-25 16:05:32,630 - INFO - Starting text cleaning.
2025-12-25 16:05:32,633 - INFO - Text cleaning completed.
2025-12-25 16:05:32,634 - INFO - Archivo cargado exitosamente. Documentos generados: 1
2025-12-25 16:05:32,634 - INFO - Step 2: Chunking document
2025-12-25 16:05:32,635 - INFO - Chunking document: Usar símboles de silencio de corchea.docx
2025-12-25 16:0

✅ Documento ingerido exitosamente!
   Total de chunks: 27

📋 Metadatos del primer chunk:
   - filename: Usar símboles de silencio de corchea.docx
   - file_type: docx
   - file_path: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx
   - source: D:\projects\Ungraph\src\data\Usar símboles de silencio de corchea.docx


### 3.4 Ingerir PDF (.pdf)

Los PDFs se procesan usando IBM Docling, que proporciona mejor extracción de texto, estructura, tablas e imágenes.


In [ ]:
# Buscar archivo PDF en la carpeta data
pdf_files = list(data_path.glob("*.pdf"))

if pdf_files:
    pdf_file = pdf_files[0]
    print(f"📥 Ingiriendo PDF: {pdf_file.name}")
    print(f"   Tamaño: {pdf_file.stat().st_size / 1024:.1f} KB")
    
    try:
        chunks_pdf = ungraph.ingest_document(
            pdf_file,
            chunk_size=1000,
            chunk_overlap=200,
            clean_text=True
        )
        print(f"✅ PDF ingerido exitosamente!")
        print(f"   Total de chunks: {len(chunks_pdf)}")
        
        # Mostrar metadatos específicos de PDF (pueden incluir page_number, document_structure, etc.)
        if chunks_pdf:
            print(f"\n📋 Metadatos del primer chunk:")
            for key, value in list(chunks_pdf[0].metadata.items())[:5]:
                print(f"   - {key}: {value}")
    except ImportError as e:
        print(f"❌ Error: {e}")
        print("   Instala langchain-docling con: pip install langchain-docling")
    except Exception as e:
        print(f"❌ Error procesando PDF: {e}")
else:
    print("⚠️  No se encontraron archivos PDF en la carpeta data")
    print("   Puedes añadir un PDF para probar esta funcionalidad")


2025-12-25 16:05:50,862 - INFO - CUDA no disponible, usando CPU para embeddings.
2025-12-25 16:05:50,867 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


📥 Ingiriendo PDF: peach-et-al-2014-qsar-modeling-of-imbalanced-high-throughput-screening-data-in-pubchem.pdf
   Tamaño: 447.4 KB


2025-12-25 16:05:53,616 - INFO - Embedding service initialized with model: sentence-transformers/all-MiniLM-L6-v2
2025-12-25 16:05:53,618 - INFO - Loading spaCy model: en_core_web_sm
2025-12-25 16:05:54,550 - INFO - spaCy model loaded successfully
2025-12-25 16:05:54,553 - INFO - Starting document ingestion: D:\projects\Ungraph\src\data\peach-et-al-2014-qsar-modeling-of-imbalanced-high-throughput-screening-data-in-pubchem.pdf
2025-12-25 16:05:54,554 - INFO - Using pattern: FILE_PAGE_CHUNK
2025-12-25 16:05:54,555 - INFO - Step 1: Loading document
2025-12-25 16:05:54,556 - INFO - Cargando archivo PDF con Docling: D:\projects\Ungraph\src\data\peach-et-al-2014-qsar-modeling-of-imbalanced-high-throughput-screening-data-in-pubchem.pdf
2025-12-25 16:05:55,364 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-25 16:05:55,394 - INFO - Going to convert document batch...
2025-12-25 16:05:55,395 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f

## Parte 4: Obtener Recomendaciones de Chunking

Antes de ingerir, podemos obtener recomendaciones sobre la mejor estrategia de chunking para cada documento.


In [ ]:
# Obtener recomendación para un archivo
markdown_file = data_path / "110225.md"

if markdown_file.exists():
    print(f"📊 Analizando: {markdown_file.name}")
    recommendation = ungraph.suggest_chunking_strategy(markdown_file)
    
    print(f"\n✅ Recomendación obtenida:")
    print(f"   Estrategia: {recommendation.strategy}")
    print(f"   Chunk size: {recommendation.chunk_size}")
    print(f"   Chunk overlap: {recommendation.chunk_overlap}")
    print(f"   Quality score: {recommendation.quality_score:.2f}")
    print(f"\n📝 Explicación:")
    print(recommendation.explanation)
    
    # Usar la recomendación para ingerir
    print(f"\n💡 Puedes usar esta recomendación al ingerir:")
    print(f"   chunks = ungraph.ingest_document(")
    print(f"       '{markdown_file.name}',")
    print(f"       chunk_size={recommendation.chunk_size},")
    print(f"       chunk_overlap={recommendation.chunk_overlap}")
    print(f"   )")
else:
    print(f"⚠️  Archivo no encontrado: {markdown_file}")


## Parte 5: Verificar Datos en el Grafo

Verifiquemos que los documentos se guardaron correctamente en Neo4j.


In [ ]:
# Verificar datos en el grafo
from src.utils.graph_operations import graph_session

driver = graph_session()
try:
    with driver.session() as session:
        # Contar nodos por tipo
        result = session.run("MATCH (f:File) RETURN count(f) as count")
        file_count = result.single()["count"]
        
        result = session.run("MATCH (p:Page) RETURN count(p) as count")
        page_count = result.single()["count"]
        
        result = session.run("MATCH (c:Chunk) RETURN count(c) as count")
        chunk_count = result.single()["count"]
        
        print("📊 Datos en el grafo:")
        print(f"   Files: {file_count}")
        print(f"   Pages: {page_count}")
        print(f"   Chunks: {chunk_count}")
        
        # Listar archivos ingeridos
        if file_count > 0:
            result = session.run("MATCH (f:File) RETURN f.filename as filename ORDER BY f.filename")
            print(f"\n📄 Archivos ingeridos:")
            for record in result:
                print(f"   - {record['filename']}")
finally:
    driver.close()


## Resumen y Mejores Prácticas

### Formatos Soportados

| Formato | Extensión | Características |
|---------|-----------|----------------|
| Markdown | .md | Estructura preservada, headers detectados |
| Texto | .txt | Detección automática de encoding |
| Word | .docx | Estructura y formato preservados |
| PDF | .pdf | Extracción avanzada con Docling (tablas, imágenes, estructura) |

### Mejores Prácticas

1. **Usar recomendaciones**: `suggest_chunking_strategy()` ayuda a elegir parámetros óptimos
2. **Ajustar chunk_size**: 
   - Textos largos: 1000-2000 caracteres
   - Textos cortos: 500-1000 caracteres
3. **Overlap razonable**: 10-20% del chunk_size (ej: 200 para chunk_size=1000)
4. **Limpiar texto**: `clean_text=True` mejora la calidad de embeddings
5. **Verificar carga**: Siempre verifica que los documentos se cargaron correctamente

### Siguiente Paso

Una vez que has ingerido documentos, continúa con:
- **2.1 Graph Pattern Construction** - Construir estructuras de grafo personalizadas
- **2.2 Smart Chunking Strategies** - Optimizar estrategias de chunking
- **3.1 Entity Extraction & Facts** - Extraer entidades y facts (fase Inference)

## Referencias

- [Guía de Ingesta](../../docs/guides/ingestion.md)
- [API Pública](../../docs/api/public-api.md)
- [Document Loader Service](../../src/infrastructure/services/langchain_document_loader_service.py)
